In [2]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

# Define the full path to your model directory
output_dir = '/Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/model_save/cls-5tasks-bs150-bs8-lr5e6-epoch100'

# Load the correct tokenizer (RobertaTokenizer) and model (T5ForConditionalGeneration)
tokenizer = RobertaTokenizer.from_pretrained(output_dir)
model = T5ForConditionalGeneration.from_pretrained(output_dir)

print("Model and tokenizer loaded successfully from:", output_dir)


Model and tokenizer loaded successfully from: /Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/model_save/cls-5tasks-bs150-bs8-lr5e6-epoch100


# Update jupyter and ipywidgets in terminal
pip install jupyter ipywidgets --upgrade

In [5]:
import pandas as pd

# Define the path to your test data
test_data_path = "/Users/kr24945/Documents/Projects/Materia science/New_polyNC/ML4Polymers/data/test/prompt-target.csv"  
test_df = pd.read_csv(test_data_path)

print("Test dataset loaded. Number of samples:", len(test_df))
print(test_df.head())  # Display first few rows to verify



Test dataset loaded. Number of samples: 2320
                                              prompt  target
0  Predict the Tg of the following SMILES: *CC(C)...  119.85
1  Predict the Tg of the following SMILES: C(C(C)...  119.85
2  Predict the Tg of the following SMILES: CC(*)(...  119.85
3  Predict the Tg of the following SMILES: *C(C*)...  119.85
4  Predict the Tg of the following SMILES: *C(C#N...  119.85


In [6]:
# Initialize a list to store predictions
predictions = []

for i, row in test_df.iterrows():
    input_text = row['input_column']  # Replace 'input_column' with the actual column name
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    
    # Generate prediction
    outputs = model.generate(input_ids)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(prediction)

    # Display progress every 10 samples (adjust as needed)
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{len(test_df)} samples.")

# Add predictions to DataFrame
test_df['predicted_output'] = predictions
print("Predictions generated and added to DataFrame.")


KeyError: 'input_column'